In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys

# Add the root folder to the module search path
# Get the current directory
current_directory = os.getcwd()

# Move two levels up (go to the parent directory of the parent directory)
two_levels_up_directory = os.path.dirname(os.path.dirname(current_directory))

print(two_levels_up_directory)

sys.path.append(two_levels_up_directory)

/home/ec2-user/SageMaker/pykoi


In [2]:
## pip install ipykernel
import pykoi

/home/ec2-user/anaconda3/envs/0731a/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Define a database, model and chatbot

To a launch a pykoi App, you only need to customize 3 components: a model, a database and a chatbot. Let's try it out!

In [3]:
qa_database = pykoi.QuestionAnswerDatabase()

In [4]:
model = pykoi.ModelFactory.create_model(
    model_source="huggingface", 
    pretrained_model_name_or_path="tiiuae/falcon-7b",
    trust_remote_code=True, ## TODO: set as default
    load_in_8bit=True
    )

[HuggingfaceModel] loading model...


Loading checkpoint shards: 100%|██████████| 2/2 [00:23<00:00, 11.75s/it]


[HuggingfaceModel] loading tokenizer...


In [5]:
chatbot = pykoi.Chatbot(model=model, feedback="vote")

### Launch the App

Now we can launch the chatbot using the model and database. Once you run the below line, you will see a Tunnel link which ends with `ngrok-free.app`. Click this link and you can see the below interface:

<p align="center">
    <img src="../image/chatbot_vote_trim_4x_crop.gif" width="75%" height="75%" />
</p>



You may need to set a ngrok token (one time) by uncomment the below line, add your [personal ngrok token](https://dashboard.ngrok.com/get-started/your-authtoken) and run this `ngrok config` line.

In [6]:
# !ngrok config add-authtoken xxx

### Add `nest_asyncio` 
Add `nest_asyncio` to avoid error such as `asyncio.run() cannot be called from a running event loop`. Since we're running another interface inside a Jupyter notebook where an asyncio event loop is already running, we'll encounter the error. (since The uvicorn.run() function uses asyncio.run(), which isn't compatible with a running event loop.)

In [8]:
# !pip install -q nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [9]:
app = pykoi.Application(debug=False, share=True)
app.add_component(chatbot)
app.run()

t=2023-08-01T04:26:32+0000 lvl=warn msg="ngrok config file found at legacy location, move to XDG location" xdg_path=/home/ec2-user/.config/ngrok/ngrok.yml legacy_path=/home/ec2-user/.ngrok2/ngrok.yml


Public URL: NgrokTunnel: "https://d7cd-34-209-105-222.ngrok-free.app" -> "http://localhost:34349"


INFO:     Started server process [21474]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:34349 (Press CTRL+C to quit)


INFO:     76.21.76.39:0 - "GET / HTTP/1.1" 200 OK
INFO:     76.21.76.39:0 - "GET /assets/index-fc1a91e8.js HTTP/1.1" 200 OK
INFO:     76.21.76.39:0 - "GET /assets/index-f75c1c12.css HTTP/1.1" 200 OK
INFO:     76.21.76.39:0 - "GET /components HTTP/1.1" 200 OK
INFO:     76.21.76.39:0 - "GET /vite.svg HTTP/1.1" 200 OK
INFO:     76.21.76.39:0 - "GET /chat/qa_table/retrieve HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [21474]


Stopping server...


### Visualize your data

Once you collect enough data via the chatbot app above, you can visualize your data via the dashboard.

In [10]:
qa_dashboard = pykoi.Dashboard(database=qa_database)

In [11]:
app = pykoi.Application(debug=False, share=True)
app.add_component(chatbot)
app.add_component(qa_dashboard)
app.run()


t=2023-08-01T04:27:23+0000 lvl=warn msg="ngrok config file found at legacy location, move to XDG location" xdg_path=/home/ec2-user/.config/ngrok/ngrok.yml legacy_path=/home/ec2-user/.ngrok2/ngrok.yml


Public URL: NgrokTunnel: "https://82f5-34-209-105-222.ngrok-free.app" -> "http://localhost:37933"


INFO:     Started server process [21474]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:37933 (Press CTRL+C to quit)


INFO:     76.21.76.39:0 - "GET / HTTP/1.1" 200 OK
INFO:     76.21.76.39:0 - "GET /assets/index-fc1a91e8.js HTTP/1.1" 200 OK
INFO:     76.21.76.39:0 - "GET /assets/index-f75c1c12.css HTTP/1.1" 200 OK
INFO:     76.21.76.39:0 - "GET /components HTTP/1.1" 200 OK
INFO:     76.21.76.39:0 - "GET /vite.svg HTTP/1.1" 200 OK
INFO:     76.21.76.39:0 - "GET /chat/qa_table/retrieve HTTP/1.1" 200 OK
INFO:     76.21.76.39:0 - "GET /chat/qa_table/retrieve HTTP/1.1" 200 OK
INFO:     76.21.76.39:0 - "GET /chat/qa_table/retrieve HTTP/1.1" 200 OK
INFO:     76.21.76.39:0 - "GET /chat/qa_table/retrieve HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [21474]


Stopping server...


You will only see one localhost link after you run the `app.run()` below. If you are on EC2 or a remote server, you can tunnel it back to your laptop via the following options:

- If you are using VSCode, check [tunnel using VSCode](https://code.visualstudio.com/docs/remote/ssh#_forwarding-a-port-creating-ssh-tunnel);
- [Directly config via EC2](https://www.opensourceforu.com/2021/09/how-to-do-reverse-tunnelling-with-the-amazon-ec2-instance/)

Now we can launch the dashboard. Click this above link and you can see the below interface:

<p align="center">
    <img src="../image/chatbot_dashboard_trim_2x.gif" width="75%" height="75%" />
</p>